In [ ]:
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

# Data prep

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

In [ ]:
# Getting the vocab size
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("".join(chars), vocab_size)

In [ ]:
stoi = { ch:i for i, ch in enumerate(chars) }
itos = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: "".join([itos[t] for t in i])

print(encode("hii there."))
print(decode(encode("hii there.")))

In [ ]:
# Encoding the entire tiny shakespeare
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# A word on training
While training these transformers we only train them on sample chunks of the text at a time, in parallel, at a fixed length. This fixed length is what we term as `block_size`

In [ ]:
# We are defining our block_size here - context length
block_size = 8

# This is going to be one training example.
# We are going to generate multiple training examples similar to the way we, given a name in our previous models.
example = train_data[:block_size+1]
print(f"{example=}")
x_example = []
for i in range(block_size):
    x_example.append(example[i].item())
    print(f"Given {x_example} predict --> {example[i+1]}")

# Retrieve sample chunk

In [ ]:
batch_size = 4
block_size = 8

def get_batch(split="train"):
    data = train_data if split == "train" else val_data
    # Gets for random start_ix
    ix = torch.randint(len(data) - block_size, (batch_size, ))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch()

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

# Quick recap:
* `Bigram` model captured the co-occurence frequencies of 2 tokens.
* The Neural Network approach was to start a random frequency matrix and forward and backprop until we got this.
* We then found that this wasn't so good and decided to use an embedding matrix instead that condensed frequency information with bigger context.
* Finally we used the wavenet idea and decomposed large contexts as composits of bigrams that propogated upwards.
This is where we are.

In [ ]:
# Reimplementing bigramNN in pytorch proper
#n_embd = 10
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # Initialize the word-frequency or weight matrix
        #self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        # Rewriting this to incorporate word embeddings C instead
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # idx = index of Xb, targets = Yb
        # This is equivalent to one-hot(xenc) @ W, which is a lookup table.
        logits = self.token_embedding_table(idx) # B, T, C
        if targets is None:
            loss = None
        else:
            # We need to reshape this to (B, C, T) as thats what pytorch uses.
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is a (B, T) and you want to take this and extend to (B, T + max_new_tokens)
        for _ in range(max_new_tokens):
            # Get logits and loss is None because targets are None.
            logits, loss = self(idx)
            # Get the logits at the Tth time dimension to predict T+1.
            logits = logits[:, -1, :]
            # Get the probabilities
            probs = F.softmax(logits, dim=-1)
            # Sample from probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # B, 1
            # Shifting to the T+1th token.
            idx = torch.cat((idx, idx_next), dim=1) # B, T+1
        return idx

In [ ]:
m = BigramLanguageModel()
out, loss = m(xb, yb)
print(out.shape, loss)

### Generating some output with the untrained model

In [ ]:
seed_ix = torch.zeros((1,1), dtype=torch.long)
print("\n--\n".join([decode(i) for i in m.generate(seed_ix, 1000).tolist()]))

### Training the model

In [ ]:
# Setting an optimizer object
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32

for _ in range(10000):
    # Get sample data
    xb, yb = get_batch("train")

    # Forward pass
    logits, loss = m(xb, yb)

    # Set grad to zero
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
seed_ix = torch.zeros((1,1), dtype=torch.long)
print("\n--\n".join([decode(i) for i in m.generate(seed_ix, 300).tolist()]))